In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon, MultiPolygon
from tqdm.notebook import tqdm
import time
from datetime import datetime

import momepy
import networkx as nx

import io
import sys

from shapely.ops import nearest_points, unary_union, cascaded_union

In [2]:
path=r'./res/ok/'

In [3]:
buil_old = gpd.read_file(path + 'bui_liv_9' + '.json', encoding='utf-8')

In [4]:
stops_800 = gpd.read_file(path + 'gdf_stops_32640_buff_9' + '.json', encoding='utf-8')

In [5]:
buff_400 = gpd.read_file(path + 'spruts_buff_400_10' + '.json', encoding='utf-8')

In [6]:
pairs = pd.read_csv('df_pairs.csv', sep=";", encoding='utf-8')

In [7]:
shrt_bui = buil_old[['osm_id', 'bui_liv_type', 'geometry']]

shrt_bui_400 = shrt_bui[shrt_bui.bui_liv_type == 'многоквартирный'].reset_index(drop=True)
shrt_bui_800 = shrt_bui[shrt_bui.bui_liv_type == 'ИЖС'].reset_index(drop=True)

In [8]:
stops_800['ped_avail_m'] = 800
buff_400['ped_avail_m'] = 400

In [9]:
%time sj_800 = gpd.sjoin(shrt_bui_800, stops_800[['stop_id', 'ped_avail_m', 'geometry']], how='left', op='intersects')
sj_800 = sj_800.drop("index_right", axis=1)
sj_800 = sj_800.drop_duplicates(subset=['osm_id']).reset_index(drop=True)

Wall time: 35.1 s


In [10]:
sj_800.head(2)

,osm_id,bui_liv_type,geometry,stop_id,ped_avail_m
0,22025513,ИЖС,"POLYGON ((50.17540 53.21847, 50.17534 53.21852...",stop__10000939,800.0
1,22025515,ИЖС,"POLYGON ((50.17303 53.21791, 50.17296 53.21798...",stop__10000939,800.0


In [11]:
np_400 = buff_400.to_numpy()
ind_stid = list(buff_400.columns).index('stop_id')
ind_geo = list(buff_400.columns).index('geometry')

np_pairs = pairs.to_numpy()
ind_one = list(pairs.columns).index('one')
ind_two = list(pairs.columns).index('two')

In [30]:
lst_inter = []
lst_stopid_one = []
lst_stopid_two = []
lst_done = []

i=0
# for i in tqdm(range(2)):

for i in tqdm(range(len(np_pairs))):
    one_stop = np_pairs[i, ind_one]
    two_stop = np_pairs[i, ind_two]
    one_pair = str(one_stop) + "^" + str(two_stop)
    one_pair2 = str(two_stop) + "^" + str(one_stop)
    
#     try:
    try:
        one_stop_poly = list(np_400[np_400[:,ind_stid] == one_stop][:,ind_geo])[0]
    except:
        one_stop_poly = []
    try:
        two_stop_poly = list(np_400[np_400[:,ind_stid] == two_stop][:,ind_geo])[0]
    except:
        two_stop_poly = []
    if ((type(one_stop_poly) == shapely.geometry.polygon.Polygon) 
        | (type(one_stop_poly) == shapely.geometry.multipolygon.MultiPolygon)):
        if ((type(two_stop_poly) == shapely.geometry.polygon.Polygon) 
            | (type(two_stop_poly) == shapely.geometry.multipolygon.MultiPolygon)):
            intr = one_stop_poly.intersection(two_stop_poly)
        else:
            intr = one_stop_poly
    else:
        if ((type(two_stop_poly) == shapely.geometry.polygon.Polygon) 
            | (type(two_stop_poly) == shapely.geometry.multipolygon.MultiPolygon)):
            intr = two_stop_poly
    #
    if ((one_pair not in lst_done) 
        & (one_pair2 not in lst_done) 
        & ((type(intr) == shapely.geometry.polygon.Polygon) 
            | (type(intr) == shapely.geometry.multipolygon.MultiPolygon))):
        lst_done.append(one_pair)
        lst_inter.append(intr)

        lst_stopid_one.append(one_stop)
        lst_stopid_two.append(two_stop)
#     except:
#         print(one_pair)
#         pass
#     #

In [91]:
def get_paired_buffs(gdf_buff, df_pairs):
    np_buff = gdf_buff.to_numpy()
    ind_stid = list(gdf_buff.columns).index('stop_id')
    ind_geo = list(gdf_buff.columns).index('geometry')

    np_pairs = df_pairs.to_numpy()
    ind_one = list(df_pairs.columns).index('one')
    ind_two = list(df_pairs.columns).index('two')
    
    lst_inter = []
    lst_stopid_one = []
    lst_stopid_two = []
    lst_done = []

    i=0
    for i in tqdm(range(len(np_pairs))):
        one_stop = np_pairs[i, ind_one]
        two_stop = np_pairs[i, ind_two]
        one_pair = str(one_stop) + "^" + str(two_stop)
        one_pair2 = str(two_stop) + "^" + str(one_stop)
        try:
            one_stop_poly = list(np_buff[np_buff[:,ind_stid] == one_stop][:,ind_geo])[0]
        except:
            one_stop_poly = []
        try:
            two_stop_poly = list(np_buff[np_buff[:,ind_stid] == two_stop][:,ind_geo])[0]
        except:
            two_stop_poly = []
        if ((type(one_stop_poly) == shapely.geometry.polygon.Polygon) 
            | (type(one_stop_poly) == shapely.geometry.multipolygon.MultiPolygon)):
            if ((type(two_stop_poly) == shapely.geometry.polygon.Polygon) 
                | (type(two_stop_poly) == shapely.geometry.multipolygon.MultiPolygon)):
                intr = one_stop_poly.intersection(two_stop_poly)
            else:
                intr = one_stop_poly
        else:
            if ((type(two_stop_poly) == shapely.geometry.polygon.Polygon) 
                | (type(two_stop_poly) == shapely.geometry.multipolygon.MultiPolygon)):
                intr = two_stop_poly
        #
        if ((one_pair not in lst_done) 
            & (one_pair2 not in lst_done) 
            & ((type(intr) == shapely.geometry.polygon.Polygon) 
                | (type(intr) == shapely.geometry.multipolygon.MultiPolygon))):
            lst_done.append(one_pair)
            lst_inter.append(intr)

            lst_stopid_one.append(one_stop)
            lst_stopid_two.append(two_stop)
    #
    
    return lst_inter, lst_stopid_one, lst_stopid_two

In [92]:
lst_inter_400, lst_stopid_one_400, lst_stopid_two_400 = get_paired_buffs(buff_400, pairs)

In [93]:
lst_inter_800, lst_stopid_one_800, lst_stopid_two_800 = get_paired_buffs(stops_800, pairs)

In [34]:
gdf_paired_400 = gpd.GeoDataFrame(geometry=lst_inter_400, 
                 data=list(zip(lst_stopid_one_400, lst_stopid_two_400)), columns=['one', 'two'])
#
gdf_paired_400.crs='epsg:4326'
gdf_paired_400['ped_avail_m'] = 400

In [95]:
gdf_paired_800 = gpd.GeoDataFrame(geometry=lst_inter_800, 
                 data=list(zip(lst_stopid_one_800, lst_stopid_two_800)), columns=['one', 'two'])
#
gdf_paired_800.crs='epsg:4326'
gdf_paired_800['ped_avail_m'] = 800

In [35]:
gdf_paired_400.head(2)

,one,two,geometry,ped_avail_m
0,stop__10001575,stop__10002125,"POLYGON ((50.18312 53.18982, 50.18311 53.18982...",400
1,stop__10001575,stop__10001577,"POLYGON ((50.18607 53.18900, 50.18607 53.18900...",400


In [99]:
%time sj_paired_400 = gpd.sjoin(shrt_bui_400, gdf_paired_400, how='left', op='intersects')
sj_paired_400 = sj_paired_400.drop("index_right", axis=1)
sj_paired_400 = sj_paired_400.drop_duplicates(subset=['osm_id']).reset_index(drop=True)

Wall time: 8.8 s


In [97]:
%time sj_paired_800 = gpd.sjoin(shrt_bui_800, gdf_paired_800, how='left', op='intersects')
sj_paired_800 = sj_paired_800.drop("index_right", axis=1)
sj_paired_800 = sj_paired_800.drop_duplicates(subset=['osm_id']).reset_index(drop=True)

Wall time: 1min 4s


In [100]:
sj_paired_400.columns

Index(['osm_id', 'bui_liv_type', 'geometry', 'one', 'two', 'ped_avail_m'], dtype='object')

In [101]:
sj_paired_400 = sj_paired_400.rename(columns={'one':'stop_id_one', 'two':'stop_id_two'})
sj_paired_400 = sj_paired_400[['osm_id', 'bui_liv_type','stop_id_one','stop_id_two', 
                 'ped_avail_m', 'geometry']]

In [98]:
sj_paired_800 = sj_paired_800.rename(columns={'one':'stop_id_one', 'two':'stop_id_two'})
sj_paired_800 = sj_paired_800[['osm_id', 'bui_liv_type','stop_id_one','stop_id_two', 
                 'ped_avail_m', 'geometry']]

In [44]:
sj_800.columns

Index(['osm_id', 'bui_liv_type', 'geometry', 'stop_id', 'ped_avail_m'], dtype='object')

In [47]:
sj_800 = sj_800.rename(columns={'stop_id':'stop_id_one'})
sj_800['stop_id_two'] = None
sj_800 = sj_800[['osm_id', 'bui_liv_type','stop_id_one','stop_id_two', 
                 'ped_avail_m', 'geometry']]

In [102]:
bui_new = sj_paired_400.append(sj_paired_800).reset_index(drop=True)

In [ ]:
# bui_new = sj_400.append(sj_800).reset_index(drop=True)

In [103]:
print(len(bui_new))
print(len(buil_old))

175376
175376


In [104]:
bui_new.head(2)

,osm_id,bui_liv_type,stop_id_one,stop_id_two,ped_avail_m,geometry
0,21759242,многоквартирный,stop__10001437,stop__10001227,400.0,"POLYGON ((50.19760 53.22646, 50.19735 53.22653..."
1,22025511,многоквартирный,NaN,NaN,NaN,"POLYGON ((50.17588 53.21841, 50.17510 53.21823..."


In [105]:
bui_new.to_file("bui_new.json", driver="GeoJSON", encoding='utf-8')

In [62]:
big_buff_paired_400 = gpd.GeoDataFrame(geometry=[cascaded_union(list(gdf_paired_400.geometry))], data=['400'], columns=['ped_avail_m'])
big_buff_paired_400.crs=gdf_paired_400.crs

In [78]:
big_buff_paired_400.to_file("big_buff_paired_400.json", driver="GeoJSON", encoding='utf-8')

In [106]:
big_buff_paired_800 = gpd.GeoDataFrame(geometry=[cascaded_union(list(gdf_paired_800.geometry))], data=['400'], columns=['ped_avail_m'])
big_buff_paired_800.crs=gdf_paired_800.crs

In [107]:
big_buff_paired_800.to_file("big_buff_paired_800.json", driver="GeoJSON", encoding='utf-8')

In [71]:
new_geo_list = [x for x in list(buff_400.geometry) if not pd.isnull(x)]
big_buff_400 = gpd.GeoDataFrame(geometry=[cascaded_union(new_geo_list)], data=['400'], columns=['ped_avail_m'])
big_buff_400.crs=buff_400.crs

In [76]:
big_buff_400.to_file("big_buff_400.json", driver="GeoJSON", encoding='utf-8')

In [73]:
big_buff_800 = gpd.GeoDataFrame(geometry=[cascaded_union(list(stops_800.geometry))], data=['800'], columns=['ped_avail_m'])
big_buff_800.crs=stops_800.crs

In [80]:
big_buff_800.to_file("big_buff_800.json", driver="GeoJSON", encoding='utf-8')

In [82]:
new_buff_400 = buff_400[~buff_400.geometry.isna()].reset_index(drop=True)

In [88]:
new_buff_400.to_file("new_buff_400.json", driver="GeoJSON", encoding='utf-8')

In [90]:
gdf_paired_400.to_file("buff_paired_400.json", driver="GeoJSON", encoding='utf-8')

In [96]:
gdf_paired_800.to_file("buff_paired_800.json", driver="GeoJSON", encoding='utf-8')